In [2]:
import numpy as np 
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from MySQLdb import connect

import os 
import json
import random
import string
from time import time
from collections import defaultdict

from utils.mch import MCHHandler

In [3]:
def get_idx(n=10):
    assert n > 1, "n should be greater than 1"
    first = random.choice(string.ascii_letters)
    rest = "".join(random.sample(string.ascii_letters + string.digits, n-1))
    return first + rest

In [6]:
# 1/cm
mus_range = {
    "skin_musp": (29.7, 48.9),
    "skin_bmie": (0.705, 2.453),
    "fat_musp": (13.7, 35.8),
    "fat_bmie": (0.385, 0.988),
    "muscle_musp": (9.8, 13.0),
    "muscle_bmie": (0.926, 2.82),
    "ijv_musp": (10, 20),
    "ijv_bmie": (10, 20),
    "cca_musp": (10, 20),
    "cca_bmie": (10, 20),
}

geo_range = {
# mm
    "geo_skin": (0.5, 1.5),
    "geo_fat": (0.5, 1.5),
    "geo_ijvr": (2.5, 10.5),
    "geo_ijvd": (2, 16),
    "geo_ccar": (2.5, 4.5),
    "geo_ccad": (5, 26),
    "geo_ijvcca": (5, 10)
}

mua_range = {
    "idx": None,
    "skin_b": (0.0012, 0.0041),
    "skin_s": (0.985, 1.0),
    "skin_w": (0.166, 0.261),
    "skin_f": (0.187, 0.277),
    "skin_m": (0.0087, 0.0165),

    "fat_f": (0.5, 1),

    "muscle_b": (0.005, 0.050),
    "muscle_s": (0.5, 0.9),
    "muscle_w": (0.5, 0.9),

    "ijv_s": (0.0, 1.0),

    "cca_s": (0.0, 1.0),
}



In [7]:
# mus
skin_musp = random.uniform(mus_range["skin_musp"][0], mus_range["skin_musp"][1])
skin_bmie = random.uniform(mus_range["skin_bmie"][0], mus_range["skin_bmie"][1])
fat_musp = random.uniform(mus_range["fat_musp"][0], mus_range["fat_musp"][1])
fat_bmie = random.uniform(mus_range["fat_bmie"][0], mus_range["fat_bmie"][1])
muscle_musp = random.uniform(mus_range["muscle_musp"][0], mus_range["muscle_musp"][1])
muscle_bmie = random.uniform(mus_range["muscle_bmie"][0], mus_range["muscle_bmie"][1])
ijv_musp = random.uniform(mus_range["ijv_musp"][0], mus_range["ijv_musp"][1])
ijv_bmie = random.uniform(mus_range["ijv_bmie"][0], mus_range["ijv_bmie"][1])
cca_musp = random.uniform(mus_range["cca_musp"][0], mus_range["cca_musp"][1])
cca_bmie = random.uniform(mus_range["cca_bmie"][0], mus_range["cca_bmie"][1])
        
# geo

geo_skin = random.uniform(geo_range["geo_skin"][0], geo_range["geo_skin"][1])
geo_fat = random.uniform(geo_range["geo_fat"][0], geo_range["geo_fat"][1])
geo_ijvd = random.uniform(geo_range["geo_ijvd"][0], geo_range["geo_ijvd"][1])
geo_ijvr = random.uniform(geo_range["geo_ijvr"][0], geo_range["geo_ijvr"][1])
while geo_skin + geo_fat > geo_ijvd - geo_ijvr:
    geo_ijvd = random.uniform(geo_range["geo_ijvd"][0], geo_range["geo_ijvd"][1])
    geo_ijvr = random.uniform(geo_range["geo_ijvr"][0], geo_range["geo_ijvr"][1])

geo_ccad = random.uniform(geo_range["geo_ccad"][0], geo_range["geo_ccad"][1])
geo_ccar = random.uniform(geo_range["geo_ccar"][0], geo_range["geo_ccar"][1])
geo_ijvcca = random.uniform(geo_range["geo_ijvcca"][0], geo_range["geo_ijvcca"][1])
while (geo_ccad - geo_ijvd)**2 + (geo_ijvcca)**2 <  (geo_ccar + geo_ijvr)**2:
    geo_ccad = random.uniform(geo_range["geo_ccad"][0], geo_range["geo_ccad"][1])
    geo_ccar = random.uniform(geo_range["geo_ccar"][0], geo_range["geo_ccar"][1])
    geo_ijvcca = random.uniform(geo_range["geo_ijvcca"][0], geo_range["geo_ijvcca"][1])



In [8]:
with open("train/template.json") as f:
    inp = json.load(f)

In [9]:
inp["skin"]["muspx"] = skin_musp
inp["skin"]["bmie"] = skin_bmie

inp["fat"]["muspx"] = fat_musp
inp["fat"]["bmie"] = fat_bmie

inp["muscle"]["muspx"] = muscle_musp
inp["muscle"]["bmie"] = muscle_bmie

inp["IJV"]["muspx"] = ijv_musp
inp["IJV"]["bmie"] = ijv_bmie
inp["CCA"]["muspx"] = cca_musp
inp["CCA"]["bmie"] = cca_bmie

inp["geometry"]["skin_thickness"] = geo_skin
inp["geometry"]["fat_thickness"] = geo_fat
inp["geometry"]["ijv_radius"] = geo_ijvr
inp["geometry"]["ijv_depth"] = geo_ijvd
inp["geometry"]["cca_radius"] = geo_ccar
inp["geometry"]["cca_depth"] = geo_ccad
inp["geometry"]["ijv_cca_distance"] = geo_ijvcca
inp["boundary"]["x_size"] = 150
inp["boundary"]["y_size"] = 150
inp["boundary"]["z_size"] = 200

In [10]:
with open("train/inptemplate.json", "w+") as f:
    json.dump(inp, f, indent=4)np

{'skin': {'n': 1.4,
  'g': 0.94,
  'muspx': 48.22294843427134,
  'bmie': 1.726165179426705},
 'fat': {'n': 1.4,
  'g': 0.94,
  'muspx': 20.814078277131763,
  'bmie': 0.9042435381978435},
 'muscle': {'n': 1.4,
  'g': 0.94,
  'muspx': 12.597990285291406,
  'bmie': 2.3639551518242095},
 'IJV': {'n': 1.4,
  'g': 0.94,
  'muspx': 18.49803738386337,
  'bmie': 19.80591435101853},
 'CCA': {'n': 1.4,
  'g': 0.94,
  'muspx': 14.559481879621172,
  'bmie': 13.522411517192968},
 'geometry': {'skin_thickness': 1.389182454180723,
  'fat_thickness': 1.186157370449629,
  'ijv_radius': 3.5728779946222256,
  'ijv_depth': 8.94258745746267,
  'cca_radius': 3.407679169349727,
  'cca_depth': 10.890177321302556,
  'ijv_cca_distance': 7.995989432088793},
 'boundary': {'x_size': 150, 'y_size': 150, 'z_size': 200}}